In [216]:
from datetime import datetime, timedelta
import pandas as pd
import requests, json
from pandas import json_normalize

In [217]:
today = datetime.now()

In [218]:
today.strftime('%Y%m%y')

tyear = today.year
tmon = today.month
tday = today.day

tmon = 4
c_date = f'{tyear:02d}-{tmon:02d}-{tday:02d}'


ser_day = datetime.strptime(c_date, '%Y-%m-%d')


In [219]:
import calendar

# 0이면 월요일, 6이면 일요일이 시작일이다.
start_day_week, end_day = calendar.monthrange(tyear, tmon)
print(start_day_week, end_day)

# 해당 달의 첫날의 datetime형식으로 변환한다.
start_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-01', '%Y-%m-%d')
end_date = datetime.strptime(f'{tyear:02d}-{tmon:02d}-{end_day:02d}', '%Y-%m-%d')
print(start_date, end_date)


5 30
2023-04-01 00:00:00 2023-04-30 00:00:00


In [220]:
start_date.strftime('%Y-%m-%d')

'2023-04-01'

In [221]:
pd.date_range(start_date.strftime('%Y-%m-%d'), periods=3)

DatetimeIndex(['2023-04-01', '2023-04-02', '2023-04-03'], dtype='datetime64[ns]', freq='D')

In [222]:
lines = []
before_month_day = 0
if start_day_week < 6:
    before_month_day = (start_date - timedelta(start_day_week+1))
    # print(before_month_day)

    lines.extend(pd.date_range(before_month_day.strftime('%Y-%m-%d'), periods=start_day_week+1))
    # for i in range(start_day_week+1):
    #     lines.append(before_month_day)
    #     before_month_day +=1

print(lines)
    


[Timestamp('2023-03-26 00:00:00'), Timestamp('2023-03-27 00:00:00'), Timestamp('2023-03-28 00:00:00'), Timestamp('2023-03-29 00:00:00'), Timestamp('2023-03-30 00:00:00'), Timestamp('2023-03-31 00:00:00')]


In [223]:
lines.extend(pd.date_range(start_date.strftime('%Y-%m-%d'), periods=end_day))

In [224]:
# lines.extend(range(1, end_day+1))
print(lines)

[Timestamp('2023-03-26 00:00:00'), Timestamp('2023-03-27 00:00:00'), Timestamp('2023-03-28 00:00:00'), Timestamp('2023-03-29 00:00:00'), Timestamp('2023-03-30 00:00:00'), Timestamp('2023-03-31 00:00:00'), Timestamp('2023-04-01 00:00:00'), Timestamp('2023-04-02 00:00:00'), Timestamp('2023-04-03 00:00:00'), Timestamp('2023-04-04 00:00:00'), Timestamp('2023-04-05 00:00:00'), Timestamp('2023-04-06 00:00:00'), Timestamp('2023-04-07 00:00:00'), Timestamp('2023-04-08 00:00:00'), Timestamp('2023-04-09 00:00:00'), Timestamp('2023-04-10 00:00:00'), Timestamp('2023-04-11 00:00:00'), Timestamp('2023-04-12 00:00:00'), Timestamp('2023-04-13 00:00:00'), Timestamp('2023-04-14 00:00:00'), Timestamp('2023-04-15 00:00:00'), Timestamp('2023-04-16 00:00:00'), Timestamp('2023-04-17 00:00:00'), Timestamp('2023-04-18 00:00:00'), Timestamp('2023-04-19 00:00:00'), Timestamp('2023-04-20 00:00:00'), Timestamp('2023-04-21 00:00:00'), Timestamp('2023-04-22 00:00:00'), Timestamp('2023-04-23 00:00:00'), Timestamp('20

In [225]:
# 빈 칸에 다음 달 날짜 구하기
nday = calendar.weekday(tyear, tmon, end_day)
next_cnt = 0
if nday == 6: next_cnt = 6
elif nday < 5: next_cnt = 5 - nday

# 마지막 날짜 구하기
lines.extend(pd.date_range((end_date+timedelta(1)).strftime('%Y-%m-%d'), periods=next_cnt))

# lines.extend(range(1, next_cnt+1))
# print(lines)

In [226]:
for n in lines[:5]:
    print(n)

2023-03-26 00:00:00
2023-03-27 00:00:00
2023-03-28 00:00:00
2023-03-29 00:00:00
2023-03-30 00:00:00


In [229]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()

- 공휴일

In [230]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey

In [231]:
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

print(res)

{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL SERVICE."},"body":{"items":{"item":[{"dateKind":"01","dateName":"1월1일","isHoliday":"Y","locdate":20230101,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230121,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230122,"seq":1},{"dateKind":"01","dateName":"설날","isHoliday":"Y","locdate":20230123,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230124,"seq":1},{"dateKind":"01","dateName":"삼일절","isHoliday":"Y","locdate":20230301,"seq":1},{"dateKind":"01","dateName":"어린이날","isHoliday":"Y","locdate":20230505,"seq":1},{"dateKind":"01","dateName":"부처님오신날","isHoliday":"Y","locdate":20230527,"seq":1},{"dateKind":"01","dateName":"대체공휴일","isHoliday":"Y","locdate":20230529,"seq":1},{"dateKind":"01","dateName":"현충일","isHoliday":"Y","locdate":20230606,"seq":1},{"dateKind":"01","dateName":"광복절","isHoliday":"Y","locdate":20230815,"seq":1},{"dateKind":"01","dateName":"추석","isHol

In [232]:
res = json.loads(res)
res.keys()

dict_keys(['response'])

In [233]:
res['response']['body']['items']['item']

[{'dateKind': '01',
  'dateName': '1월1일',
  'isHoliday': 'Y',
  'locdate': 20230101,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230121,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230122,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '설날',
  'isHoliday': 'Y',
  'locdate': 20230123,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230124,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '삼일절',
  'isHoliday': 'Y',
  'locdate': 20230301,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '어린이날',
  'isHoliday': 'Y',
  'locdate': 20230505,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '부처님오신날',
  'isHoliday': 'Y',
  'locdate': 20230527,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '대체공휴일',
  'isHoliday': 'Y',
  'locdate': 20230529,
  'seq': 1},
 {'dateKind': '01',
  'dateName': '현충일',
  'isHoliday': 'Y',
  'locdate': 20230606,
  'seq': 1},
 {'dateKind': '01',
  'd

In [234]:
holidays = json_normalize(res['response']['body']['items']['item'])
holidays

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
5,01,삼일절,Y,20230301,1
6,01,어린이날,Y,20230505,1
7,01,부처님오신날,Y,20230527,1
8,01,대체공휴일,Y,20230529,1
9,01,현충일,Y,20230606,1


- 24절기

In [235]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/get24DivisionsInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
res.keys()

dict_keys(['response'])

In [236]:
json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']]

,dateKind,dateName,isHoliday,locdate
0,03,소한,N,20230106
1,03,대한,N,20230120
2,03,입춘,N,20230204
3,03,우수,N,20230219
4,03,경칩,N,20230306
5,03,춘분,N,20230321
6,03,청명,N,20230405
7,03,곡우,N,20230420
8,03,입하,N,20230506
9,03,소만,N,20230521


- 기념일

In [237]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getAnniversaryInfo?_type=json&numOfRows=80&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
dt = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])])


In [238]:
dt

,dateKind,dateName,isHoliday,locdate,seq
0,01,1월1일,Y,20230101,1
1,01,설날,Y,20230121,1
2,01,설날,Y,20230122,1
3,01,설날,Y,20230123,1
4,01,대체공휴일,Y,20230124,1
...,...,...,...,...,...
64,02,아동학대예방의 날,N,20231119,1
65,02,소비자의 날,N,20231203,1
66,02,무역의 날,N,20231205,1
67,02,자원봉사자의 날,N,20231205,2


- 잡절


In [239]:
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getSundryDayInfo?_type=json&numOfRows=50&solYear='
base_url += str(tyear) + '&ServiceKey=' + apikey
result = requests.get(base_url)
if result.status_code == 200:
    res = result.text

res = json.loads(res)
json_normalize(res['response']['body']['items']['item'])

,dateKind,dateName,isHoliday,locdate,seq
0,04,정월대보름,N,20230205,1
1,04,한식,N,20230406,1
2,04,단오,N,20230622,1
3,04,초복,N,20230711,2
4,04,중복,N,20230721,1
5,04,말복,N,20230810,1
6,04,칠석,N,20230822,1


- 한 번에 구하기

In [240]:
with open('static/key/holidays.txt') as f:
    apikey = f.read()
    
f_list = ['getRestDeInfo', 'getAnniversaryInfo', 'get24DivisionsInfo', 'getSundryDayInfo']
base_url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/' #getSundryDayInfo?_type=json&numOfRows=50&solYear='
param1 = '?_type=json&numOfRows=80&solYear='
holidays = None
for name in f_list:
    url = base_url + name + param1 + str(tyear) + '&ServiceKey=' + apikey
    res = ''

    try:
        result = requests.get(url) 
        if result.status_code == 200:
            print('200')
            res = result.text
    except:
        pass

    if res: 
        res = json.loads(res)
        holidays = pd.concat([holidays, json_normalize(res['response']['body']['items']['item'])[['dateKind', 'dateName', 'isHoliday', 'locdate']] ])

200
200
200
200


In [241]:
holidays.head()

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,20230101
1,01,설날,Y,20230121
2,01,설날,Y,20230122
3,01,설날,Y,20230123
4,01,대체공휴일,Y,20230124


In [242]:
holidays.isna().sum().sum()

0

In [243]:
holidays['locdate'] = holidays['locdate'].astype(str)

In [244]:
# type(list(holidays['locdate'])[0])

In [245]:
# '20230101' in list(holidays['locdate'])

In [246]:
# holidays['locdate'].to_datetime()
holidays['locdate'] = pd.to_datetime(holidays['locdate'], format='%Y%m%d')

In [254]:
for dd in holidays['locdate'][:5]:
    print(dd)

2023-01-01 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00


In [255]:
holidays

,dateKind,dateName,isHoliday,locdate
0,01,1월1일,Y,2023-01-01
1,01,설날,Y,2023-01-21
2,01,설날,Y,2023-01-22
3,01,설날,Y,2023-01-23
4,01,대체공휴일,Y,2023-01-24
...,...,...,...,...
2,04,단오,N,2023-06-22
3,04,초복,N,2023-07-11
4,04,중복,N,2023-07-21
5,04,말복,N,2023-08-10


In [249]:
try:
    nindex = holidays.index[holidays.dateName=='기독탄신일'][0]
    holidays.iloc[nindex, 1] = '크리스마스'
except:
    pass



In [250]:
holidays[holidays.dateName=='크리스마스']

,dateKind,dateName,isHoliday,locdate
16,01,크리스마스,Y,2023-12-25


In [259]:
holidays.locdate[:5]

0   2023-01-01
1   2023-01-21
2   2023-01-22
3   2023-01-23
4   2023-01-24
Name: locdate, dtype: datetime64[ns]

In [261]:
l_dt = pd.DataFrame(lines, columns=['locdate'])
l_dt.head()

,locdate
0,2023-03-26
1,2023-03-27
2,2023-03-28
3,2023-03-29
4,2023-03-30


In [296]:
l_dt['day'] = l_dt.locdate.apply(lambda x: int(x.strftime('%d')))


In [297]:
result_dt = pd.merge(l_dt, holidays, how='left', on='locdate')
result_dt.dateKind.fillna('05', inplace=True)
result_dt.dateName.fillna('', inplace=True)
result_dt.isHoliday.fillna('N', inplace=True)
result_dt

,locdate,day,dateKind,dateName,isHoliday
0,2023-03-26,26,05,,N
1,2023-03-27,27,05,,N
2,2023-03-28,28,05,,N
3,2023-03-29,29,05,,N
4,2023-03-30,30,05,,N
5,2023-03-31,31,05,,N
6,2023-04-01,1,05,,N
7,2023-04-02,2,05,,N
8,2023-04-03,3,02,4·3희생자 추념일,N
9,2023-04-04,4,05,,N


In [270]:
res_list = result_dt.to_dict('records')
